In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as const 
from matplotlib.colors import LogNorm
from matplotlib import gridspec

## original data
Is located on hemera at

```
/bigdata/hplsim/aipp/Jeyhun/khi/part_rad/metrics/002_900_1001_yci9wlgu
```


In [ ]:
omega_rad = np.logspace(np.log10(0.1), np.log10(200), 512) # in [omega_pe]

In [ ]:
UNIT_ENERGY = 7.42701e-11
UNIT_TIME = 1.79e-16

UNIT_MASS = 8.26366e-28
UNIT_LENGTH = 5.36628e-08
UNIT_SPEED = 2.99792e+08

typical_weigthing = 907.159

In [ ]:
UNIT_MASS * UNIT_SPEED / typical_weigthing / (const.electron_mass * const.c)

In [ ]:
data_gpu = {}
N_GPUs = 96
for i in range(N_GPUs):
    data_gpu[i] = np.load("../data_last_02/data_gpu_{}_tindex_950.npz".format(i))


In [ ]:
for i in data_gpu[0].keys():
    print(i)

In [ ]:
for i in range(N_GPUs):
    plt.plot(omega_rad, np.exp(data_gpu[i]["rad_gt"]) , label="{:02d}".format(i))
    
    if ((i-3) % 4) == 0:
        plt.fill_betweenx([1e-10, 1e2], 40 , 60, color="pink", alpha=0.4)
        plt.xscale("log")
        plt.yscale("log")

        plt.xlabel(r"$\omega /\omega_\mathrm{pe}$", fontsize=18)
        plt.xticks(fontsize=14)

        plt.ylabel(r"$\frac{\mathrm{d}^2 I}{\mathrm{d} \omega \mathrm{d} \Omega} \, \mathrm{[arb.u.]}$", fontsize=18)
        plt.yticks(fontsize=14)

        plt.legend(loc=3)

        plt.show()

In [ ]:
for i in range(N_GPUs):
    plt.hist(data_gpu[i]["ps_gt"][:, 0], bins=64, range=(-0.3, +0.3), label="{:02d}".format(i), alpha=0.5)

    if ((i-3) % 2) == 0:
        plt.yscale("log")
        plt.legend(loc=5)
        plt.show()

In [ ]:
gpu_index_toDetector = 23 # 21 #26# 1 # #26# 
gpu_index_awayFromDetector = 59 ###37 # 14 #13 
gpu_index_vortex = 18 #32 #31 #27 # 20 #18 <-- #5# 4#3 
fig = plt.figure(figsize=(8,6))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 2.0]) 

gs1 = gs[1].subgridspec(2, 1, wspace=0, hspace=0)

plt.subplot(gs[0])

plt.text( 1.3e2, 0.3e2, "(a)", fontsize=18)

plt.text( 0.12, 0.4e-6, "solid: ground truth    dotted: ML prediction", fontsize=12)


#plt.title("radiation spectra", fontsize=16)
#plt.fill_betweenx([1e-10, 1e30], 40 , 60, color="gray", alpha=0.8)
#plt.fill_between([0.1, 2e2], 1e-5 , 2e-2, color="gray", alpha=0.4)

plt.plot(omega_rad, np.exp(data_gpu[gpu_index_toDetector]["rad_gt"]), color="C0")
plt.plot(omega_rad, np.exp(data_gpu[gpu_index_toDetector]["rad_pred"]), ":", color="C0", alpha=0.5)

plt.plot(omega_rad, np.exp(data_gpu[gpu_index_awayFromDetector]["rad_gt"]), color="C1")
plt.plot(omega_rad, np.exp(data_gpu[gpu_index_awayFromDetector]["rad_pred"]), ":", color="C1", alpha=0.5)

plt.plot(omega_rad, np.exp(data_gpu[gpu_index_vortex]["rad_gt"]), color="C2")
plt.plot(omega_rad, np.exp(data_gpu[gpu_index_vortex]["rad_pred"]), ":", color="C2", alpha=0.5)


plt.plot([5], [5], color="gray", label="PIConGPU")
plt.plot([5], [5], ":", color="gray", alpha=0.5, label="ML prediction")

#plt.legend(loc=3, fontsize=14)
         
plt.xscale("log")
plt.xlim(0.1, 200)
plt.yscale("log")
plt.ylim(1e-7, 1.5e+3)

#plt.xlabel(r"$\omega \, [\omega_\mathrm{pe}]$", fontsize=18)
plt.xlabel("frequency in units of plasma frequency", fontsize=18)
plt.xticks(fontsize=14)
#plt.ylabel(r"$\frac{\mathrm{d}^2 I}{\mathrm{d} \omega \mathrm{d} \Omega} \, \mathrm{[arb.u.]}$", fontsize=18)
plt.ylabel("radiation\nspectrum", fontsize=18)
plt.yticks(fontsize=14)




##########
##########

plt.subplot(gs1[0])
plt.text( 0.265, 0.6e2, "(b)", fontsize=18)
plt.text( 0.0, 0.6e2, "ground truth: PIConGPU", fontsize=18, ha="center")


#plt.title("momentum distribution and prediction", fontsize=16)
# to detector
tmp = np.histogram(data_gpu[gpu_index_toDetector]["ps_gt"][:, 0],
                   bins=128, range=(-0.3, +0.3), 
                   density=True)
plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C0", alpha=0.5, label="to detector")

#tmp = np.histogram(data_gpu[gpu_index_toDetector]["ps_pr_inn"][:, 0],
#                   bins=128, range=(-0.3, +0.3), 
#                   density=True)
#plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C0", alpha=1.0, hatch='.', facecolor="None")

# away from detector
tmp = np.histogram(data_gpu[gpu_index_awayFromDetector]["ps_gt"][:, 0],
                   bins=128, range=(-0.3, +0.3), 
                   density=True)
plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C1", alpha=0.5, label="away from detector")

#tmp = np.histogram(data_gpu[gpu_index_awayFromDetector]["ps_pr_inn"][:, 0],
#                   bins=128, range=(-0.3, +0.3), 
#                   density=True)
#plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C1", alpha=1.0, hatch='//', facecolor="None")


# KHI vortex
tmp = np.histogram(data_gpu[gpu_index_vortex]["ps_gt"][:, 0],
                   bins=128, range=(-0.3, +0.3), 
                   density=True)
plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0.0, lw=1, color="C2", alpha=0.5, label="KHI vortex")

#tmp = np.histogram(data_gpu[gpu_index_vortex]["ps_pr_inn"][:, 0],
#                   bins=128, range=(-0.3, +0.3), 
#                   density=True)
#plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C2", alpha=1.0, hatch='\\', facecolor="None")


plt.yscale("symlog")
plt.xscale("symlog")

plt.ylim(1e-2, 2e2)
plt.xticks([])#[-0.3, -0.2, -0.1, 0.0, +0.1, +0.2, +0.3], ["-0.3", "-0.2", "-0.1", "0.0", "+0.1", "+0.2", "+0.3"])
plt.xlim(-0.3, +0.3)



#plt.ylabel(r"$\frac{\mathrm{d}Q}{\mathrm{d} p_x} \, \mathrm{[arb.u.]}$", fontsize=18)
plt.ylabel("charge\ndensity", fontsize=18)

plt.yticks(fontsize=14)


plt.legend(loc=8, fontsize=14)


###############
###############



plt.subplot(gs1[1])
plt.text( 0.265, 0.6e2, "(c)", fontsize=18)
plt.text( 0.0, 0.6e2, "ML prediction", fontsize=18, ha="center")

#plt.title("momentum distribution and prediction", fontsize=16)
# to detector
#tmp = np.histogram(data_gpu[gpu_index_toDetector]["ps_gt"][:, 0],
#                   bins=128, range=(-0.3, +0.3), 
#                   density=True)
#plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C0", alpha=0.5, label="to detector")

tmp = np.histogram(data_gpu[gpu_index_toDetector]["ps_pr_inn"][:, 0],
                   bins=128, range=(-0.3, +0.3), 
                   density=True)
plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C0", alpha=0.5,)# hatch='.', facecolor="None")





# away from detector
#tmp = np.histogram(data_gpu[gpu_index_awayFromDetector]["ps_gt"][:, 0],
#                   bins=128, range=(-0.3, +0.3), 
#                   density=True)
#plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C1", alpha=0.5, label="away from detector")

tmp = np.histogram(data_gpu[gpu_index_awayFromDetector]["ps_pr_inn"][:, 0],
                   bins=128, range=(-0.3, +0.3), 
                   density=True)
plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C1", alpha=0.5, )#hatch='//', facecolor="None")






# KHI vortex
#tmp = np.histogram(data_gpu[gpu_index_vortex]["ps_gt"][:, 0],
#                   bins=128, range=(-0.3, +0.3), 
#                   density=True)
#plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0.0, lw=1, color="C2", alpha=0.5, label="KHI vortex")

tmp = np.histogram(data_gpu[gpu_index_vortex]["ps_pr_inn"][:, 0],
                   bins=128, range=(-0.3, +0.3), 
                   density=True)
plt.fill_between((tmp[1][1:] + tmp[1][:-1])/2., tmp[0], 0, lw=1, color="C2", alpha=0.5, )#hatch='\\', facecolor="None")


plt.yscale("symlog")
plt.xscale("symlog")

plt.ylim(1e-2, 2e2)
plt.xticks([-0.3, -0.2, -0.1, 0.0, +0.1, +0.2, +0.3], ["-0.3", "-0.2", "-0.1", "0.0", "+0.1", "+0.2", "+0.3"])
plt.xlim(-0.3, +0.3)

plt.xlabel(r"momentum in the direction of the detector", fontsize=18)
plt.xticks(fontsize=14)

plt.ylabel("charge\ndensity", fontsize=18)
plt.yticks(fontsize=14)

#plt.fill_between([1, 2], [1,2], 0, lw=1, color="white", alpha=1.0, hatch='\\\\', facecolor="None", label=" ")
#plt.fill_between([1, 2], [1,2], 0, lw=1, color="gray", alpha=0.5,  label="PIConGPU")
#plt.fill_between([1, 2], [1,2], 0, lw=1, color="gray", alpha=1.0, hatch='\\\\', facecolor="None", label="ML prediction")


#plt.legend(loc=9, fontsize=14)

plt.tight_layout()

#fig.subplots_adjust(wspace=0.15, left=0.10, right=0.95)

plt.savefig("spec_to_phaseSpace_inversINN_simple_new03.pdf")

plt.show()

In [ ]:
plt.text?

In [ ]:
for data in [data_gpu_8, data_gpu_9, data_gpu_10, data_gpu_11, data_gpu_13, data_gpu_14]:

    plt.hist2d(data["ps_gt"][:, 0], data["ps_gt"][:, 1], 
               bins=256, range=((-0.3, +0.3),(-0.1, +0.1)), density=True,
               norm=LogNorm(), cmap=plt.cm.inferno, rasterized=True)
    #plt.colorbar()


    plt.xlabel(r"$p_x / (m_e \cdot c)}$", fontsize=18)
    plt.xticks(fontsize=14)

    plt.ylabel(r"$p_y / (m_e \cdot c)}$", fontsize=18)
    plt.yticks(fontsize=14)
    plt.colorbar()
    plt.show()